In [1]:
import pandas as pd
import os
import json
from openpyxl import load_workbook
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.styles import Border, Side

In [2]:
# !pip install openpyxl

In [ ]:
def list_directories(path):
    directories = [f"{path}/{d}" for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) 
                   and ".ipynb_checkpoints" not in d]
    return directories

# Get List of experiments for each dataset

In [4]:
results_dataset_paths = ["results"]

In [5]:
experiments_paths_by_dataset = {}

for dataset_path in results_dataset_paths:
    experiments_paths_by_dataset[dataset_path] = list_directories(dataset_path)
experiments_paths_by_dataset

{'results': ['results/1-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.1',
  'results/10-Delta approach, Lr=1e-05, epochs=100, dropout=0.2',
  'results/11-Delta approach, Lr=1e-05, epochs=200, dropout=0.1',
  'results/2-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.25',
  'results/3-Data Loss + Physics Loss, lambda=10, Lr=0.0001, epochs=100, dropout=0.1',
  'results/4-Data Loss + Physics Loss, lambda=10, Lr=0.0001, epochs=100, dropout=0.25',
  'results/5-Delta approach, Lr=0.0001, epochs=100, dropout=0.1',
  'results/6-Delta approach, Lr=0.0001, epochs=100, dropout=0.2',
  'results/7-Delta approach, Lr=0.001, epochs=100, dropout=0.1',
  'results/8-Delta approach, Lr=0.001, epochs=100, dropout=0.2',
  'results/9-Delta approach, Lr=1e-05, epochs=100, dropout=0.1']}

# Get metrics of each model in each experiment in each dataset result

In [6]:
all_datasets_metrics = {}
for dataset_path in experiments_paths_by_dataset.keys():
    dataset_results_metrics = []
    for experiment_path in experiments_paths_by_dataset[dataset_path]:
        # get all models in the experiment
        models_paths = list_directories(experiment_path)
        
        # iterate over models paths and get their metrics
        for model_path in models_paths:
            # open json file with metrics
            metrics_filepath = f"{model_path}/metrics.json"
            with open(metrics_filepath, 'r', encoding='utf-8') as file:
                metrics = json.load(file)    

            metrics_to_store = {
                "Experiment settings": experiment_path.split("/")[-1],
                "Model": metrics_filepath.split("/")[-2],
                "RMSE (Validation set)": metrics["validation set RMSE"],
                "RMSE": metrics['test set loss (RMSE)'],
                "MSE": metrics['test set loss (MSE)'],
                "MAE": metrics['test set loss (MAE)'],
            }

            dataset_results_metrics.append(metrics_to_store)

    # store metrics of all experiments for the current dataset
    all_datasets_metrics[dataset_path] = dataset_results_metrics
print(all_datasets_metrics)

{'results': [{'Experiment settings': '1-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.1', 'Model': 'Physics', 'RMSE (Validation set)': 0.11910554282367229, 'RMSE': 0.12949894797056913, 'MSE': 0.019650885567534714, 'MAE': 0.1131453599780798}, {'Experiment settings': '1-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.1', 'Model': 'Physics-and-Transformer', 'RMSE (Validation set)': 0.1627283997833729, 'RMSE': 0.2840119659900665, 'MSE': 0.09821933070197701, 'MAE': 0.2529686190187931}, {'Experiment settings': '1-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.1', 'Model': 'Transformer', 'RMSE (Validation set)': 0.13039994947612285, 'RMSE': 0.15696772374212742, 'MSE': 0.05124309260863811, 'MAE': 0.13692331463098525}, {'Experiment settings': '10-Delta approach, Lr=1e-05, epochs=100, dropout=0.2', 'Model': 'Physics', 'RMSE (Validation set)': 0.11910554282367229, 'RMSE': 0.12949894797056913, 'MSE': 0.019650885567534714, 'MAE

# Store metrics of all experiments in the corresponding dataset result directory

In [7]:
for dataset_directory in all_datasets_metrics.keys():
    current_dataset_metrics = all_datasets_metrics[dataset_directory]

    # transform current dataset metrics to dataframe and store it
    df = pd.DataFrame(current_dataset_metrics)
    # print(df)
    file_path = f"{dataset_directory}/metrics_of_all_experiments.xlsx"
    df.to_excel(file_path, index=False, engine='openpyxl')

    # apply conditional formatting to last 3 columns (because they contain the metric values). 
    # This section of code came from AI assistant: https://chatgpt.com/share/67a8fb48-ae30-800b-b9c4-44caa979d630
    # Load the workbook and the active sheet
    wb = load_workbook(file_path)
    ws = wb.active
    
    # Columns to apply formatting (assumes last 3 columns are numeric metrics)
    columns = [ws.cell(row=1, column=i+1).column_letter for i in range(len(df.columns)-4, len(df.columns))]
    
    for col in columns:
        ws.conditional_formatting.add(
            f"{col}2:{col}{len(df)+1}",
            ColorScaleRule(start_type="min", start_color="00FF00",  # Green
                           mid_type="percentile", mid_value=50, mid_color="FFFF00",  # Yellow
                           end_type="max", end_color="FF0000")  # Red
        )

    # Add outher border each 3 rows
    # Define border style
    thick_border = Border(
        left=Side(style='thick'),
        right=Side(style='thick'),
        top=Side(style='thick'),
        bottom=Side(style='thick')
    )
    
    # Apply outer border for each group of 3 rows
    for i in range(2, len(df) + 2, 3):
        for col in range(1, len(df.columns) + 1):
            ws.cell(row=i, column=col).border = Border(top=thick_border.top, left=thick_border.left, right=thick_border.right)
            ws.cell(row=i+1, column=col).border = Border(left=thick_border.left, right=thick_border.right)
            ws.cell(row=i+2, column=col).border = Border(bottom=thick_border.bottom, left=thick_border.left, right=thick_border.right)

    # Adjust column widths based on content length
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter
        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        ws.column_dimensions[col_letter].width = max_length + 2  # Add some padding
    
    # Save the modified workbook
    wb.save(file_path)

In [8]:
# Get latex string

# Convert to LaTeX string
latex_code = df.to_latex(
    index=False,        # Don’t include the DataFrame index
    escape = True,
    columns=["Experiment settings", "Model", "RMSE (Validation set)"], # remove columns that are not important for the report
    longtable=True,     # Tells pandas to use \begin{longtable} instead of \begin{tabular}
    na_rep="-",         # How to display NaNs or missing data
    column_format='p{5cm} | c | c',
    
    caption="Metrics of Transformer-Encoder",
    label="tab:metrics-transformer-encoder",                 # LaTeX label for cross-referencing
)

with open(f"{dataset_directory}/metrics_of_all_experiments.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)

# print(latex_code)